<a class="anchor" id="toc"></a>
# Содержание

[Описание проекта](#about)

1. [Откроем таблицу и изучим общую информацию о данных](#open)


2. [Предобработка данных](#preprocessing)    
    2.1. [Удаление пропусков](#preprocessing)    
    2.2. [Обработка аномальных значений](#processing_anomaly)    
    2.3. [Удаление пропусков (продолжение)](#delete_skip)    
    2.4. [Изменение типов данных](#processing_type)    
    2.5. [Обработка дубликатов](#processing_duplicates)    
    2.6. [Категоризация данных](#category)
    
    
3. [Исследуем данные и ответим на вопросы](#study_values)    
    3.1. [Есть ли зависимость между количеством детей и возвратом кредита в срок?](#study_values)    
    3.2. [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#family_status)    
    3.3. [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#income_level)    
    3.4. [Как разные цели кредита влияют на его возврат в срок?](#target)    
    3.5. [Приведите возможные причины появления пропусков в исходных данных.](#why_skip)    
    3.6. [Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.](#why_median)
    

4. [Общий вывод](#conclusion)

    [Портрет идеального клиента](#portret)

<a class="anchor" id="about"></a>
## Описание проекта

### Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<a class="anchor" id="open"></a>
### 1. Откроем таблицу и изучим общую информацию о данных

Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`.

In [2]:
import pandas as pd

In [ ]:
data = pd.read_csv('/datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран.

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<a class="anchor" id="preprocessing"></a>
### 2. Предобработка данных
#### 2.1. Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Небольшое отступление**

Мы нашли одинаковое количество пропусков в двух столбцах. Чтобы понять лучше природу этих пропусков определим, принадлежат ли эти пропуски одним и тем же записям. Выведем суммарное количество строк, которые содержат пропущенные значения либо в колонке `total_income`, либо `days_employed`:

In [6]:
data.loc[(data['total_income'].isna()) | (data['days_employed'].isna())]['total_income'].isna().count()

2174

Количество пропусков совпадает с найденными ранее значениями по обеим колонкам, соответственно пропуски по двум колонкам принадлежат одним и тем же строкам.

---

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

<a class="anchor" id="processing_anomaly"></a>
#### 2.2. Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.

In [8]:
data['days_employed'] = data['days_employed'].abs()

---

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправим такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

--- 

Выведем перечень уникальных значений столбца `children`.

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [12]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [13]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

<a class="anchor" id="delete_skip"></a>
#### 2.3. Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [14]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [15]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<a class="anchor" id="processing_type"></a>
#### 2.4. Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [16]:
data['total_income'] = data['total_income'].astype(int)

<a class="anchor" id="processing_duplicates"></a>
#### 2.5. Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [17]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [18]:
data.duplicated().sum()

71

In [19]:
data = data.drop_duplicates()

<a class="anchor" id="category"></a>
#### 2.6. Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000 — `'B'`.

In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [22]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

In [23]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [24]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

<a class="anchor" id="study_values"></a>
### 3. Исследуем данные и ответим на вопросы
#### 3.1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

Сформируем сводную таблицу с количеством клиентов, должников и долей должников.

In [25]:
(
    data
    .pivot_table(index='children', values=['total_income', 'debt'], 
                 aggfunc={'total_income': 'count',
                     'debt': [sum, 'mean']},  
                 fill_value=0)
    .round(3)
    .rename(columns={'count' : 'client', 'sum' : 'debtors', 'mean' : 'ratio'})
)

debt         total_income
          ratio debtors       client
children                            
0         0.075    1063        14091
1         0.092     444         4808
2         0.095     194         2052
3         0.082      27          330
4         0.098       4           41
5         0.000       0            9

На первый взгляд зависимость не очевидна, но проанализируем количественно наши выборки: определим сколько людей в каждой группе и какую часть они составляют от общего количества заемщиков:
 - в колонке `count` - количество клиентов;
 - в колонке `sum` - количество должников;
 - в колонке `mean` - доля должников.

In [26]:
# посчитаем общее количество заемщиков:
all_people = data['total_income'].count()

# найдем количество заемщиков в каждой группе:
share_of_all = data.groupby('children').agg({'total_income' : 'count'})

# посчитаем долю заемщиков каждой группы от всех заемщиков:
share_of_all.assign(share = (share_of_all['total_income'] / all_people) * 100).round(2)

,total_income,share
children,,
0,14091,66.06
1,4808,22.54
2,2052,9.62
3,330,1.55
4,41,0.19
5,9,0.04


Мы видим, что группы, имеющие 3, 4 и 5 детей, в сумме составляют менее 2% от общего количества исследуемых (380 человек), что недостаточно для формирования каких-либо точных выводов по ним и сравнения их с первыми тремя группами. Данные выборки не репрезентативны и не могут достаточно точно отразить характер генеральной совокупности своих групп. Поэтому для анализа будем опираться на первые 3 группы: 
1. Заемщики без детей;
2. Заемщики с 1 ребенком;
3. Заемщики с 2 детьми.

---
**Вывод:** Существует прямая зависимость между количеством детей и возвратом кредита в срок: с увеличением количества детей возвращаемость кредита в срок снижается. Можно сделать вывод, что многодетным семьям труднее выплачивать кредит (возвращаемость займа в срок 90,8% с одним ребенком в семье и 90,5% с двумя соотетственно), нежели бездетным (92,5%).

___

<a class="anchor" id="family_status"></a>
#### 3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Отобразим в сводной число заемщиков, разделенных на разные группы по семейному положению и число должников. Также укажем долю должников от общего числа заемщиков:

In [27]:
(
    data
    .pivot_table(index='family_status', values=['total_income', 'debt'], 
                 aggfunc={'total_income': 'count',
                     'debt': [sum, 'mean']},  
                 fill_value=0)
    .round(3)
    .rename(columns={'count' : 'client', 'sum' : 'debtors', 'mean' : 'ratio'})
)

debt         total_income
                       ratio debtors       client
family_status                                    
Не женат / не замужем  0.098     273         2796
в разводе              0.071      84         1189
вдовец / вдова         0.066      63          951
гражданский брак       0.093     385         4134
женат / замужем        0.076     927        12261

Оценим размер выборок получившихся групп:

In [28]:
# найдем количество заемщиков в каждой группе:
share_of_all = data.groupby('family_status').agg({'total_income' : 'count'})

# посчитаем долю заемщиков каждой группы от всех заемщиков:
share_of_all.assign(share = (share_of_all['total_income'] / all_people) * 100).round(2)

,total_income,share
family_status,,
Не женат / не замужем,2796,13.11
в разводе,1189,5.57
вдовец / вдова,951,4.46
гражданский брак,4134,19.38
женат / замужем,12261,57.48


Самая маленькая выборка состоит из 951 человека, что составляет 4.5% от генеральной совокупности. Данные выборки можно считать репрезентативной.

Разобьем клиентов по возрастным категориям:
- до 18 лет - дети;
- с 18 до 44 - молодые;
- с 44 до 60 - средний возраст;
- после 60 - престарелые.

In [29]:
def categorize_dob_years(years):
    try:
        if 0 <= years <= 17:
            return 'child'
        elif 18 <= years <= 44:
            return 'young'
        elif 45 <= years <= 60:
            return 'avg'
        elif years >= 61:
            return 'old'
    except:
        pass

Добавим столбец `dob_years_category` с возрастными категориями клиентов:

In [30]:
data['dob_years_category'] = data['dob_years'].apply(categorize_dob_years)

Выведем количество вдовцов/вдов по возрастным категориям:

In [31]:
(
    data
    .loc[data['family_status'] == 'вдовец / вдова']
    .groupby(data['dob_years_category'])['dob_years_category']
    .count()
)

dob_years_category
avg      500
child      5
old      348
young     98
Name: dob_years_category, dtype: int64

---
<a class="anchor" id="short_conclusion"></a>
**Промежуточный вывод:** 
- Самыми ответственными заемщиками оказалить вдовцы/вдовы. 
- Больше всех задерживают патежи не женатые/не замужние. 

Также из получившихся результатов можно сделать вывод, что люди, имеющие штамп в паспорте (вдовцы/вдовы, люди в разводе либо женатые/замужние), более ответствено относятся к выплате кредита, нежели более свободные заемщики (не женатые/не замужние или состоящие в гражданском браке). 

Тот факт, что люди, которые состоят в официальном браке, лучше выплачивают кредит можно объяснить тем, что работающей семейной паре проще погасить долги из семейного бюджета, который в основном складывается из двух зарплат.

Объяснить высокую выплачиваемость у вдовцов/вдов можно следующим предположением: подавляющее большинство вдовцов/вдов (89%) старше 45 лет. Людям такого возраста (особенно престарелым, которые составляют 37%) одобряют займы на меньшие сроки и при наличии финансовых возможностей для погашения кредита. Следовательно, данным заемщикам могут не одобрить крупную сумму займа и их ежемесячный платеж будет составлять посильную сумму для выплат. 
___

<a class="anchor" id="income_level"></a>
#### 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим 4 колонки:
1. Категории дохода заемщиков;
2. Доля тех, кто имел просрочку, от общего числа заемщиков данной группы;
3. Количество заемщиков, которые имели просрочку по кредиту;
4. Общее число заемщиков.

In [32]:
(
    data
    .pivot_table(index='total_income_category', values=['total_income', 'debt'], 
                 aggfunc={'total_income': 'count',
                     'debt': [sum, 'mean']},  
                 fill_value=0)
    .round(3)
    .rename(columns={'count' : 'client', 'sum' : 'debtors', 'mean' : 'ratio'})
)

debt         total_income
                       ratio debtors       client
total_income_category                            
A                      0.080       2           25
B                      0.071     354         5014
C                      0.085    1353        15921
D                      0.060      21          349
E                      0.091       2           22

Определим величины выборок, чтобы понять, можно ли "в лоб" сравнивать полученные доли:

In [33]:
# найдем количество заемщиков в каждой группе:
share_of_all = data.groupby('total_income_category').agg({'total_income' : 'count'})

# посчитаем долю заемщиков каждой группы от всех заемщиков:
share_of_all.assign(share = (share_of_all['total_income'] / all_people) * 100).round(2)

,total_income,share
total_income_category,,
A,25,0.12
B,5014,23.51
C,15921,74.64
D,349,1.64
E,22,0.10


Выборка категорий A, D и E составляют менее 2% (396 человек) от всей генеральной совокупности, что недостаточно для сравнения их с другими групами. При анализе будем опираться на показатели категорий B и C.

---
**Вывод:** Существует прямая зависимость между уровнем дохода и возвратом кредитов в срок: чем выше доход - тем выше возвратность. Напомним, что к категори 'C' относятся доходы 50001–200000р, а к категории 'B' — 200001–1000000р. Таким образом в категорию 'С' попали клиенты, с довольно низким заработком, что сказывается на возвращаемости долгов.
___

<a class="anchor" id="target"></a>
#### 3.4. Как разные цели кредита влияют на его возврат в срок?

Сгруппируем всех заемщиков по целям, на которые был взят кредит:
- операции с автомобилем;
- операции с недвижимостью;
- получение образования;
- проведение свадьбы,

добавим число из них с задолжностями по кредиту и вычислим какой процент должников есть в каждой группе: 

In [34]:
(
    data
    .pivot_table(index='purpose_category', values=['total_income', 'debt'], 
                 aggfunc={'total_income': 'count',
                     'debt': [sum, 'mean']},  
                 fill_value=0)
    .round(3)
    .rename(columns={'count' : 'client', 'sum' : 'debtors', 'mean' : 'ratio'})
)

debt         total_income
                          ratio debtors       client
purpose_category                                    
операции с автомобилем    0.093     400         4279
операции с недвижимостью  0.073     780        10751
получение образования     0.093     369         3988
проведение свадьбы        0.079     183         2313

Полученные значения очень похожи. Проверим размер каждой группы чтобы понимать, насколько корректно их сравнивать:

In [36]:
# найдем количество заемщиков в каждой группе:
share_of_all = data.groupby('purpose_category').agg({'total_income' : 'count'})

# посчитаем долю заемщиков каждой группы от всех заемщиков:
share_of_all.assign(share = (share_of_all['total_income'] / all_people) * 100).round(2)

,total_income,share
purpose_category,,
операции с автомобилем,4279,20.06
операции с недвижимостью,10751,50.40
получение образования,3988,18.70
проведение свадьбы,2313,10.84


Чаще всего (в 50% случаев) кредит берется для операции с недвижимостью, реже всего - на проведение свадьбы (11%).

Размеры выборок каждой группы достаточно крупные (минимальная равна 11%, 2313 человек), чтобы их можно было сравнивать и по сравнению делать выводы.

---
**Вывод:** Меньше всего просречек по кредитам у группы, берущая в долг на операции с недвижимостью. На втором месте с очень небольшой разницей оказались те, кто берет кредит на проведение свадьбы. И больше всего просрочек у двух групп, которые брали в долг на операцию с автмобилем и получение образования. 

Такая картина может объясняться тем, что кредиты на свадьбы обычно составляют относительно небольшие суммы, а на недвижимость - наоборот, но на более длительный срок, что отражается небольшой финансовой нагрузкой на кредитора. В то же время кредиты на авто обычно составляют большие суммы, но на меньший срок, нежели у недвижимости, а кредит на учебу берут люди, которые, как правило, не имеют должного образования, либо недовольны своей зарплатой, либо и того и другого и, соответственно, кредиты создают на заемщиков повышенную финансовую нагрузку, что сказывается на выплачиваемости долгов в срок.
___

<a class="anchor" id="why_skip"></a>
#### 3.5. Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Пропуски нам встретились в двух колонках: в `days_employed` и в `total_income`. Ранее,
в [небольшом отступлении](#section_id), мы определили, что пропуски по обеим колонкам принадлежат одним и тем же записям. Учитывая, что клиент не может не указать свой доход при оформлении кредита, а характер пропусков в данних количественных переменных носит полностью случайный характер, можно предположить, что возможными причинами возникновения проблемы могли быть технические проблемы: ошибки при сборе информации или выгрузке данных с сервера.

Также, в колонке `days_employed` содержались отрицательные значения. Это может быть связано с форматом хранения данных и в принципе расчета дней (например если за точку отсчета берется настоящая дата, а количество дней рассчитывается как разность между сегодняшним днем и датой устройства на работу).

<a class="anchor" id="why_median"></a>
#### 3.6. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Пропуски в количественных переменных обычно заполняют характерными значениями: средним арифметическим или медианой. В данных могут быть выдающиеся значения, которые сместят среднее в большую или меньшую сторону.

<a class="anchor" id="conclusion"></a>
### 4. Общий вывод

- При исследовании надежности заемщиков былы изучены полученные данные, обработаны пропуски и дубликаты, удалены аномальные значения. 
- Следует обратиться к разработчикам, которые выгружают данные и уведомить их, что существует проблема связанная с пропусками в данных и отрицательными значениями.
- Сообщить банку, что необходимо обратить внимание дубликатам в записях и продумать вопрос категоризации данных (например цели кредита и количества детей), чтобы облегчить в дальнейшем их обработку.
- Было выявлено:
    - что многодетным семьям труднее выплачивать кредит (процент должников 9,2% с одним ребенком в семье и 9,5% с двумя соотетственно), нежели бездетным (7,5%); 
    - люди, состоящие в официальном браке (вдовцы/вдовы, люди в разводе либо женатые/замужние), более ответствено относятся к выплате кредита - процент должников составляет до 7,1%, нежели более свободные заемщики (не женатые/не замужние или состоящие в гражданском браке) - процент должноков которых составляет до 9,8%; 
    - чем выше доход - тем выше возвратность: у категорий людей с доходами 50001–200000р процент должников составляет 8,5%, у категории с доходами 200001–1000000р. - 7,1%; 
    - Меньше всего просречек по кредитам у группы, берущая в долг на операции с недвижимостью - процент должников 7,3%. На втором месте с очень небольшой разницей оказались те, кто берет кредит на проведение свадьбы - 7,9%. И больше всего просрочек у двух групп, которые брали в долг на операцию с автмобилем и получение образования - 9,3%.

Даные инсайты и замечания помогут более качественно принимать решения при выдаче кредитов и повысить качество и скорость дальнейших или повторных исследований.

<a class="anchor" id="portret"></a>
**Портрет идеального клиента:** без детей, в официальном браке с суммарный доходом выше 200000р, берущий кредит на жилье или свадьбу.

[К содержанию](#toc)